In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/repos/Epilepsy_Microglia
%pip install -q -r requirements.txt

In [1]:
# project name
project = "Epilepsy_Microglia"
dataset = "thrupp"
gse_accession = "GSE153807"
link = "https://www.ncbi.nlm.nih.gov/geo/download/?acc=GSE201048&format=file"

In [6]:
# environment setting
from src.env_utils import detect_env, get_paths
env = detect_env()
paths = get_paths(project)
print(env)
print(paths["base"])

code-server
/home/neuro_demo_research/data_from_drive/Epilepsy_Microglia


In [8]:
from pathlib import Path
import os
if(env == "colab"):
    download_dir = Path(f"/content/downloads/{dataset}")
elif(env == "code-server"):
    download_dir = Path(f"/home/neuro_demo_research/downloads/{dataset}")
else:
    download_dir = Path(f"~/neuro_demo_research/downloads/{dataset}").expanduser()

raw_dir = paths["raw"] / dataset

os.makedirs(download_dir, exist_ok=True)
os.makedirs(raw_dir, exist_ok=True)